CBOW 모델 실습

In [5]:
import numpy as np
import tensorflow as tf

In [3]:
# 전처리 함수화

def preprocess(text):
  text = text.lower()
  text = text.replace(".", " .")
  words = text.split(' ')
  word_to_id = {}
  id_to_word={}

  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id]=word
 
  #1
  corpus = np.array([word_to_id[w] for w in words])
  
  #2
  # corpus = []
  # for w in words:
  #   corpus.append(word_to_id[w])
  # corpus = np.array(corpus)

  return corpus,word_to_id,id_to_word

In [6]:
text = "You say goodbye and I say hello."
corpus,word_to_id,id_to_word = preprocess(text)
vocab_size = len(word_to_id)

* Trainning Set 구성
* 모델구현
* 학습코드 구현

In [8]:
train_set = []
for i in range(1,len(corpus)-1):
  target_id = corpus[i]
  context = [corpus[i-1],corpus[i+1]]
  train_set.append((context,target_id))


In [9]:
train_set

[([0, 2], 1), ([1, 3], 2), ([2, 4], 3), ([3, 1], 4), ([4, 5], 1), ([1, 6], 5)]

In [11]:
# x =train_set[0][0]
# y =train_set[0][1]

#1 
X=np.array([data[0] for data in train_set])
y=np.array([data[1] for data in train_set])

#2
# y=[]
# for data in train_set:
#   y.append(data[1])
# y=np.array(y)

In [13]:
y = tf.keras.utils.to_categorical(y,num_classes=vocab_size) # one-hot 인코딩

In [14]:
y

array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [16]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,3,input_shape=(2,)) # input shape - outshape 를 지정해주면 알아서 지정(?), one-hot 인코딩 안해도 됌.

In [17]:
model = tf.keras.Sequential(
    [embedding_layer,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(vocab_size,activation='softmax')]
)

In [18]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 3)              21        
_________________________________________________________________
global_average_pooling1d (Gl (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 7)                 28        
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer="adam", loss="categorical_crossentropy")
history = model.fit(X,y,batch_size=X.shape[0])


1/1 [==============================] - 0s 2ms/step - loss: 1.9502


In [21]:
embedding_layer(word_to_id['you'])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.00504509, 0.01374583, 0.01071877], dtype=float32)>

In [20]:
embedding_layer.get_weights()

[array([[ 0.00504509,  0.01374583,  0.01071877],
        [-0.02976842,  0.01300974,  0.02155201],
        [ 0.04851675, -0.01369502,  0.02421463],
        [ 0.04586797, -0.00537508, -0.02590821],
        [-0.04898423, -0.02668343, -0.0135177 ],
        [-0.01799102, -0.00968038, -0.02301788],
        [-0.0311203 , -0.00206784,  0.01368542]], dtype=float32)]

# Skip gram

In [22]:
train_set = []
for i in range(1,len(corpus)-1):
  target_id = corpus[i]
  context = [corpus[i-1],corpus[i+1]]
  train_set.append((context,target_id))

In [23]:
X=np.array([data[0] for data in train_set])
y=np.array([data[1] for data in train_set])

In [24]:
y = tf.keras.utils.to_categorical(y,num_classes=vocab_size) # one-hot 인코딩

In [25]:
y

array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.]], dtype=float32)